# COVID-19 járványadatok vizualizációja
<a target="_blank" href="https://colab.research.google.com/github/reguly/aiappdev/blob/main/CovidVisualization.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [4]:
# Letölthető fájl az adatokkal - magyar statisztikák
!wget https://users.itk.ppke.hu/~regiszo/korona_hun.xlsx
# Letölthető tömörített fájl az adatokkal - szimulációs eredmények
!wget https://users.itk.ppke.hu/~regiszo/covid_data.zip
!unzip covid_data.zip

--2024-04-06 06:52:26--  https://users.itk.ppke.hu/~regiszo/korona_hun.xlsx
Resolving users.itk.ppke.hu (users.itk.ppke.hu)... 193.225.109.33
Connecting to users.itk.ppke.hu (users.itk.ppke.hu)|193.225.109.33|:443... connected.


HTTP request sent, awaiting response... 200 OK
Length: 1309893 (1.2M) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: 'korona_hun.xlsx.4'

korona_hun.xlsx.4   100%[===================>]   1.25M  1.55MB/s    in 0.8s    

2024-04-06 06:52:28 (1.55 MB/s) - 'korona_hun.xlsx.4' saved [1309893/1309893]

--2024-04-06 06:52:28--  https://users.itk.ppke.hu/~regiszo/covid_data.zip
Resolving users.itk.ppke.hu (users.itk.ppke.hu)... 193.225.109.33
Connecting to users.itk.ppke.hu (users.itk.ppke.hu)|193.225.109.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1230950 (1.2M) [application/zip]
Saving to: 'covid_data.zip'

covid_data.zip      100%[===================>]   1.17M  1.92MB/s    in 0.6s    

2024-04-06 06:52:28 (1.92 MB/s) - 'covid_data.zip' saved [1230950/1230950]

Archive:  covid_data.zip
   creating: covid_data/
  inflating: covid_data/series_2_9.csv  
  inflating: covid_data/series_1_10.csv  
  inflating: covid_data/series_2_8.c

Első lépések: készítsünk ábrát a fertőzöttek számának alakulásáról
- X tengelyen dátumok
- “Lyukak” az adatban – (hétvége, heti jelentés) “interpoláljunk” (töltsük ki adatokkal a lyukakat amik a lyuk előtti és azt követő értékek közt vannak)


Készítsünk további ábrákat, diagramokat
1. Aktív fertőzöttek – gyógyultak – elhunytak egymáson (stacked)
2. Új fertőzöttek száma, és a 7 napos mozgóátlag
3. Kórházi ápoltak száma
4. Új elhunytak száma
- Kórházi ápoltak milyen aránya? Mit mond el ez a kórházi ellátásról, vagy az adott variánsról?
- Vessük ezt össze az aktív fertőzöttek száma és a kórházi ápoltak arányával. Mennyire voltak súlyosak az egyes variánsok?
5. Hatósági házi karanténban lévők száma
- Aktív fertőzöttek arányában? Mikor, hogyan változhattak a szabályok?
6. Növekedési faktor – vonal 1.0-nál
- 1 alatt csökken, 1 felett nő a fertőzés terjedése
7. Beoltottak száma (Új, második)
8. Tesztek száma, mozgóátlaga, tesztpozitivitás
- Milyen kérdéseket vet fel az így kijött tesztpozitivitás? Valószínű, hogy a pozitív tesztek száma jól leírja a járvány helyzetét?
9. Az 5 hullám egymásra vetítve: kórházi ápoltak, halálozási arány, tesztek száma
- Próbáljuk meg egy ábrán, 3 alábrával
- Kézzel ki kell bogarászni mettől-meddig tartottak a hullámok



## PanSim - ágens-alapú járványmodell
PPKE ITK-n fejlesztettük a járvány előrejelzésére, intézkedések hatásának vizsgálatára
179500 ágens – nem a 9.730.000 magyar populáció – normalizálni kell!
## Táblázatos kimenet
- Oszlopok magyarázata (100-ik sor környéke): https://github.com/khbence/pansim/blob/main/matlab/Readme/Compute_beta.txt
- Stochasztikus szimuláció – 10-20 futtatás, ezek átlaga, szórása fontos
- 2 adatsor: series_1_\*.csv, series_2_\*.csv
- Szimulációk 2020 szeptember 23-ával kezdődnek

### Feladatok
1. Olvassunk be 1 CSV-t, nézzük meg a szerkezetét
- Ábrázoljuk az I1 görbét, a megfelelő dátumokkal a vízszintes tengelyen
2. Olvassuk be mind a 10 bemeneti fájlt az első adatsorból
- Ábrázoljuk mind a 10 táblázat I1-es oszlopát egy görbén
- Számoljunk minden mezőnek átlagot, szórást (kivéve a sorszám mezőt)
- Ábrázoljuk az I1 görbe átlagát, és köré halványan a szórást (fill_between – átlag +/- szórás)
3. Ábrázoljuk a következőket
- Kórházi ápoltak száma: I5_h + I6_h + R_h oszlopok
- Napi új fertőzöttek száma: NI oszlop
- Kumulatív fertőzések száma – NI-ből számítva
- Az egyes hullámok a lakosság hány %-át fertőzhették át?
- Aktív fertőzöttek száma: E+I1+I2+I3+I4+I5_h+I6_h
- Különböző variánsok százalékos aránya: MUT0…5
- Eddigi Halálozások száma: D1
- Új halálozások száma – D1-ből számítva
- Tesztek száma: T+P2, pozitív tesztek száma P1+P2, tesztpozitivitási arány
- Karanténozottak száma: Q, karanténozott és beteg QT, beteg de nem karanténozott NQ
4. Ahol van összevethető hivatalos adat, tegyük rá az ábrára
- Figyelem: nem ugyanott kezdődik és ér véget az adat. Kumulatív jellegű adatnál máshol is kezdődik! Csak a szimulált időszakra tegyük rá a hivatalosat
5. Olvassuk be a másik adatsort is, egy ábrán látszódjon a két adatsor és a hivatalos adat
- Próbáljunk meg valami “szép” megoldást adni arra, hogy több görbe is legyen egy ábrán
- Számítsuk ki a modell hibáját a két adatsorra a kórházi terhelés adathoz képest

